In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pprint import pprint
import missingno
import pandas_profiling as pp
import re # regular expressions

In [ ]:
!pip install pandas_profiling # basina "!" isareti koyarak paket indirebiliriz

In [ ]:
# Setting up inline plotting using jupyter notebook "magic"
%matplotlib inline # sadece jupiter icin gecerli. plt.show() yazmaya gerek yok artik.

# With this magic in place, all plots created in code cells will automatically be displayed inline.

In [ ]:
# Dataframe de tum sutunlari gostermek, maksimum sutun sayisini belirlemek
pd.set_option("display.max_columns", None)
pd.set_option('display.max_colwidth', -1)  # Display all values
pd.set_option('display.max_rows', 50) # gosterilecek satir sayisini belirlemek

# 🔸 ~ DATA SET DOSYALARI ~ 🔸

In [ ]:
titanic = pd.read_csv('C:\\Users\\ihbay\\Desktop\\DATA SETS\\titanic.csv')

In [ ]:
estate = pd.read_csv('C:\\Users\\ihbay\\Desktop\\DATA SETS\\hurriyetemlak_dirty.csv')

In [ ]:
medal = pd.read_csv('C:\\Users\\ihbay\\Desktop\\DATA SETS\\all_medalists.csv')

In [ ]:
# listeleri goze daha hos gozukecek sekilde yazdirir
products = ['sofas', 'convertible sofas', 'love seats', 'recliners', 'sofa beds', 'sofas', 'convertible sofas', 
            'love seats', 'recliners', 'sofa beds', 'sofas', 'convertible sofas', 'love seats', 'recliners', 'sofa beds']
from pprint import pprint
pprint(products)

In [ ]:
# eksik degerleri grafikte gostermek
missingno.matrix(titanic, figsize = (30,10)) # figsize: The size of the figure to display

In [ ]:
# hangi sutundan kac tane veri eksik oldugunu dongu ile bulmak
missing_vals = {}
titanic_length = len(titanic)
for column in titanic.columns:
    total_column_values = titanic[column].value_counts().sum()
    missing_vals[column] = titanic_length-total_column_values
missing_vals

In [ ]:
# hangi sutundan kac tane veri eksik oldugunu kisaca bulmak
titanic.isnull().sum()

In [ ]:
titanic["sex"]=titanic["sex"].str.replace("man", "wo")
titanic["cabin"]=titanic["cabin"].str.replace('\d+', '') # tum sayilari yer degistiriyor

In [ ]:
# tekrar eden satirlari silme
titanic.drop_duplicates(inplace=True)

In [ ]:
titanic.shape

In [ ]:
# farkli sutunlari secerek ayni cins degerlerin "ortalamalarini" alarak yeni bir dataframe olusturmak
titanic.pivot_table (index='age', columns='sex') 

# eger pazantez icinde values='fare' seklinde bir sutunu secersek sadece o bilgileri gosterir
# aggfunc='count' frekans tablosunu verirr. yani ayni cins degerlein ortalamasini almak yerine kac tane olduklarini verir.
# margins=True, aggfunc=sum  degerlerin toplamini verir

In [ ]:
# pivot table ile karmasik tablolar olusturmak
titanic = medal.pivot_table(index='NOC', values='Athlete', columns='Medal', aggfunc='count')

# satirlari toplayip yeni bir sutun olusturmak
titanic['totals'] = titanic.sum(axis='columns')

# dataframe i bir sutunu baz alarak siraya sokmak
titanic.sort_values(by='totals', ascending=False)

In [ ]:
# 've' kosulunu kullanarak ilgili satirlari bulmak
medal[(medal.Event_gender == 'W')& (medal.Gender == 'Men')]

In [ ]:
# iki sutun secerek ayni satirdaki tekrarlari tespit etmek
medal[['Event_gender', 'Gender']].drop_duplicates()

In [ ]:
# ulkelerin ozgun sporlardan kac tanesinden madalya aldiklarinin sayisini bulma
medal.groupby('NOC').Sport.nunique().sort_values(ascending=False)

In [ ]:
# farkli kosullarda gruplama yapmak ve ozgun degere gore siralamak
country = medal.loc[((medal.Edition <= 1988) &  (medal.Edition >= 1952)) & medal.NOC.isin(['USA', 'URS'])].groupby('NOC')
country.Sport.nunique().sort_values(ascending=False)

In [ ]:
# bir sutunda yer alan sadece bir degere ait tum verileri listelemek 
medal[medal.NOC == 'TUR']

In [ ]:
# alinan madalyalara gore cubuk grafigi cizme
medals = medal[medal.NOC == 'TUR'].groupby(['Edition', 'Medal'])['Athlete'].count()

# simdi grafigi cizebilmek icin 'unstack' metodu ile madalyalari sutunlara dagitacagiz
medals = medals.unstack(level='Medal')

medals.plot()
medals.plot.area()
plt.show()


In [ ]:
medal.drop_duplicates()

In [ ]:
titanic.info()

In [ ]:
# deger cesidini category yapmak daha az yer kaplar ve daha hizli islem yapilir.
titanic['sex'] = titanic['sex'].astype('category')

In [ ]:
# bir sutundaki ozgun degerleri tespit etmek. 
titanic['sex'].unique()

In [ ]:
# sutunlardaki degerlerin sayilari, ortalamalarini veya toplamlarini belli degerlere gore gruplandirmak
titanic.groupby('sex')[['age', 'survived']].mean() 

# en sona sum() yazarak toplamlari veya mean() ile ortalamalari alinabilir
# mean() ozelligi ile degerlerin yuzdesi hesaplanabilir. ornekte erkeklerin yaklasik sadece % 20 si kurtulmus.
# max() ile en buyuk degerlere ulasilabilir

In [ ]:
# gruplandirdiktan sonra birden fazla ozelligi uygulayabiliriz
titanic.groupby('sex')[['age', 'survived']].agg(['max', 'count', 'mean'])

In [ ]:
# dataframe de birden fazla index varsa "level" ozelligi kullanilir
titanic.groupby(level=['age','sex'])

In [ ]:
titanic.groupby('age').count()

# 🔸 ~ DATA FRAME INDEX ISLEMLERI ~ 🔸

In [ ]:
# data frame de index degistirme. 
titanic.index  
# data frame de index olarak 0 dan baslayan sayilari atama
titanic.index = range(len(titanic)) # data frame de tek bir indexi degistirmezsin. tumunu birden degistirmek gerekir. 

In [ ]:
# her hangi bir sutunu index olarak atamak 
titanic.index = titanic.age

In [ ]:
# iki sutunu index olarak atama. (Multi index)
titanic = titanic.set_index(['sibsp', 'age'])

In [ ]:
# multi index te ikinci siradaki indexin bir degerini iceren tum satirlari listelemek
titanic.loc[(slice(None), 8.00), :]

In [ ]:
# index leri siralamak
titanic = titanic.sort_index()

In [ ]:
# data frame de index leri buyuk veya kucuk harf yapma. tek satir komutla...
titanic.index = [name.upper()  for name in titanic.index]

In [ ]:
# data frame de index lere veya sutunlara ana baslik atama
titanic.index.name = 'NAMES'
titanic.columns.name = 'INFO'

In [ ]:
titanic

In [ ]:
# multi index li data frameden istedigimiz satirlari ve sutunlari secmek
titanic.loc[(1, 14.00), ('fare', 'body')]

# 🔸 ~ DATA FRAME DE HUCRE SECME ~ 🔸

In [ ]:
# sadece bu satir ve sutunlari alir
titanic.loc['Allison, Master. Hudson Trevor':'Allison, Mr. Hudson Joshua Creighton', 'sex':'ticket']  

In [ ]:
titanic['sex']['67'] 

In [ ]:
titanic.name[67]

In [ ]:
titanic.loc[67, 'name']

In [ ]:
titanic.iloc[67,2]

In [ ]:
titanic[['age', 'sex']].head()  # sadece bu sutunlari getirir

In [ ]:
titanic.iloc[1:6, 2:5]  # belli satir ve sutun araliklarini secmek

In [ ]:
titanic.iloc[[2,4,5], [3,6]] # satir ve sutunlari tek tek de secebiliriz

In [ ]:
titanic[['sex']] # sutunu cift koseli parantez icine alirsak dataframe olusur

In [ ]:
titanic['sex'] # sutunu tek koseli parantez icine alirsak data frame degil, series olusur

In [ ]:
 titanic[(titanic.age >= 20) & (titanic.sex == 'male')] # her iki kosulu da saglayan degerler                   

In [ ]:
titanic[(titanic.age >= 20) | (titanic.sex == 'male')] # herhangi bir kosulu saglayan degerler

In [ ]:
titanic.loc[:, titanic.isnull().any()]  # icinde NaN degeri iceren sutunlari getirir. Yani tum degerleri tam olanlari.

In [ ]:
titanic.loc[:, titanic.notnull().all()]  #  Hic NaN degeri olmayan sutunlari getirir

In [ ]:
titanic.dropna(how='all', inplace=True) # tum degerleri NaN olan satirlari dataframeden cikarir 

In [ ]:
titanic.dropna(how='any', inplace=True) # her hangi bir degeri NaN olan satirlari dataframeden cikarir 
# inplace=True yapmazsak sadece tespit yapiyor ama dusurmuyor

In [ ]:
titanic.dropna(thresh=6, inplace=True, axis='columns') 
# 6 tane dolu varsa silme. 6 nin altinda veri bulunuyorsa bir satirda o satiri komple siler
# satir ya da sutunlarda belli bir sayinin altinda veri varsa o satir ve sutunlari dataframe den cikarir 

In [ ]:
titanic.isna()

In [ ]:
titanic.shape

In [ ]:
# iki dataframe in esit olup olmadigini kontrol etme
titanic.equals(users)

# 🔸 ~ Data frame de kullanilabilecek faydali metod ve komutlar ~ 🔸

In [ ]:
titanic.head()

In [ ]:
titanic.info()

In [ ]:
titanic.describe(include='all') # parantez icinr include='all' yazarsan numeric disindaki tum degerleri dikkate alir. 
# tum sutunlari gosterir ve daha detayli bilgi derir.

In [ ]:
titanic.describe() # numeric degerleri baz alir.

In [ ]:
titanic.columns

In [ ]:
titanic.shape

In [ ]:
titanic.columns.value_counts()

In [ ]:
titanic.dtypes

In [ ]:
# Mevcut durumda bulundugumuz dizinin yolunu/adresini ogrenmek
# dosya adrsini yazarken aralara tek '\' konuldugunda hata veriyor. cift konulmali ya da adresin basina 'r' harfi konulmali.
pwd

In [ ]:
# uzerinde calisilan dizini degistirme
%cd "C:\1.BASAK BURCU\24. IT\DATA SCIENCE"

In [ ]:
# bulundugumuz dizindeki tum icerigi listelemek
ls 

In [ ]:
# birden fazla satırı "komut dışı" bırakmak 
ALT tuşuna basarak  ( Büyük + işareti çıkacaktır)  istediğiniz satırları seçiniz.
Daha sonra sol tuşla seçilen satırların soluna gelip "#"  işaretini yazınız.

In [ ]:
# Listelerden dataframe olusturma
# (LISTE => DICTIONARY => DATAFRAME)
cities = ['Austin', 'Dallas', 'Austin', 'Dallas']
signups = [7, 12, 3, 5]
visitors = [139, 237, 326, 456]
weekdays = ['Sun', 'Sun', 'Mon', 'Mon']
list_labels = ['city', 'signups', 'visitors', 'weekday']

# listeleri tek bir listede birlestiriyoruz. her bir liste dataframe de bir sutuna donusecektir.
list_cols = [cities, signups, visitors, weekdays]

# Liste isimleri (bunlar dataframe de sutun ismi olacaktir.) ile listeleri birlestirip yeni bir liste olusturuyoruz. 
# Bu yeni liste tuple lardan olusmaktadir. 
zipped = list(zip(list_labels, list_cols))

# Artik son listemizi dictionary ye cevirebiliriz.
data = dict(zipped)

# Son olarak dictionary yi dataframe ceviriyoruz ve islem tamamlaniyor.  
users = pd.DataFrame(data)

print(users)
data

In [ ]:
# DataFrame de sutun ismini degistirmek
gapminder.columns = ['country', 'year', 'life_expectancy']

In [ ]:
# DataFrame deki tum verileri (1.sutun haric) bir fonksiyon araciligi ile kontrol ettirmek.
# Fonksiyon degerlerinin >= 0 olup olmadigini kontrol ediyor.
assert g1800s.iloc[:, 1:].apply(check_null_or_valid, axis=1).all().all()

In [ ]:
# DataFrame de hucre degerleinin kac kere tekrar ettiklerini bulmak
titanic['country'].value_counts().keys().tolist() : sadece hucre degerleri 
titanic['country'].value_counts().tolist() : sadece kac tane olduklari
titanic['country'].value_counts() : hucre degerleri ve kac tane olduklari birlikte

In [ ]:
# Bir degeri tersine cevirmek
Basina ~ simgesini koymak yeterli

In [ ]:
# DataFrame dosyasini csv veya xls olarak kaydetmek
titanic.to_csv('file_clean.csv', index=False)
titanic.to_excel('file_clean.xlsx', index=False)

In [ ]:
# Bir dictionaryi dataframe cevirmek (value ler liste seklinde olmali). 
# Key ler dataframe lerin sutun isimleri olarak atanacaktir.
titanic = pd.DataFrame({"a":['1'], 'b':['2']})

In [ ]:
# dataframe de bir sutunun degerlerini grafikte gosterme
plt.plot(titanic['tmdbId'].values)
plt.show()

In [ ]:
# dataframe de tum sutunlari grafikte gosterme
plt.plot(titanic)
plt.show()

In [ ]:
# dataframe de x ve y eksenlerine sutun atayarak farkli grafikler cizdirme, baslik, x ve y eksenine isim verme
# kind = 'scatter', 'box', 'hist' olabilir
titanic.plot(x='imdbId', y='tmdbId', kind='hist')
plt.xlabel('x-ekseni')
plt.ylabel('y-ekseni')
plt.title('Baslik')

In [ ]:
# Sutunlari ayri grafikte gosterme
# bunun icin subplots=True  yaoilmasi gerekir
titanic[['imdbId', 'tmdbId']].plot(kind='box', subplots=True)
plt.show()

In [ ]:
# dataframe de bir sutunu silmek
del titanic['yeni']

In [ ]:
# dataframe de sutun isimlerini degistirmek. bu komutta tum sutun isimlerini yazmak gerekir.
titanic.columns = ['deneme1', 'title', 'deneme3']

In [ ]:
# csv dosyasini dataframe'e donustururken sutun isimlerini atama
col_names = ['year', 'month', 'day', 'dec_date', 'sunspots','definite']
titanic = pd.read_csv('C:\\Users\\ihbay\\Desktop\\movies.csv', header = None, names=col_names)
titanic.iloc[10:20, :] 

In [ ]:
# dataframe de bir sutundaki bir degerin kac tane oldugunu bulmak
titanic[titanic.origin == 'Asia']

In [ ]:
# dataframe de hangi sutunda kac veri oldugunu ogrenmek
titanic.count()

In [ ]:
titanic.iloc['16']

In [ ]:
# zaman serilerinde dosyayi okurken index olarak tarihleri baz alma ve tarihleri belli bor formata sokma. 
# bu sekilde okundugu takdirde tahihler kullanilarak dataframe her turlu formata sokulabilir

titanic = pd.read_csv('C:\\Users\\ihbay\\Desktop\\movies.csv', index_col='Date', parse_dates=True)
titanic.loc['2010-Aug-01']

In [ ]:
# dataframe de bir sutunda yer alan bir anahtar kelimeyi veya ifadeyi iceren kac deger oldugunu bulmak
# ancak sadece string degerler icin gecerli
titanic['tmdbId'].str.contains('ware').sum() 

In [ ]:
# Convert the 'Date' column into a collection of datetime objects: titanic.Date
titanic.Date = pd.to_datetime('Date')

In [ ]:
gapminder = pd.read_csv('gapminder.csv', index_col=['Year','region','Country'] ).sort_index()
def spread(series):
    return series.max() - series.min()
gapminder.groupby(level=['Year', 'region']).agg({'population':'sum', 'child_mortality':'mean', 'gdp':spread})

In [ ]:
titanic.head()

In [ ]:
# dataframe i fonksiyon ve dictionary kullanarak gruplandirma
titanic1 = pd.read_csv('C:\\Users\\ihbay\\Desktop\\DATA SETS\\titanic.csv', index_col=['name', 'sex', 'pclass']).sort_index()
def spread(series):
    return series.max() - series.min()
titanic1.groupby(level=['pclass', 'sex']).agg({'age':'mean', 'survived':'sum', 'fare':spread})

In [ ]:
# bosluklari ortancayi kullanrak dolduran bir fonksiyon yazma
def impute_median(series):
    return series.fillna(series.median())

In [ ]:
# bir sutunun Nan olan degerlerini o sutunun ortalamasi ile doldurmak
titanic['body'].fillna(titanic['body'].mean(), inplace=True)

In [ ]:
# her hangi bir sutunun bir ifadeyi icermesine gore bulup onlarin ortalalmasini, toplamini vs almak
titanic.loc[titanic['home.dest'].str.contains('New'), 'age'].mean()

In [ ]:
titanic.fillna(method='pad', inplace=True)

In [ ]:
titanic.info()

In [ ]:
titanic.head()

In [ ]:
# Read the CSV file into a DataFrame: sales
sales = pd.read_csv('sales.csv', index_col='Date', parse_dates=True)

# Group sales by 'Company': by_company
by_company = sales.groupby('Company')

# Compute the sum of the 'Units' of by_company: by_com_sum
by_com_sum = by_company['Units'].sum()
print(by_com_sum)

# Filter 'Units' where the sum is > 35: by_com_filt
by_com_filt = by_company.filter(lambda g: g['Units'].sum()>35)

In [ ]:
# belli sutun degerlerinin ortalamasi belli bir sayidan buyuk olan satirlari listelemek
titanic.groupby('pclass').filter(lambda g: g['age'].mean()>30)

In [ ]:
# yasi 10 un altinda olanlarin kurtulma yuzdesinin ortalamasi 
titanic.groupby(titanic['age']<10)['survived'].mean()

# true / False yerine isimlerini yazamak icin .map() metodu kullanilir
titanic.groupby((titanic['age']<10).map({True:'under 10', False:'over 10'}))['survived'].mean()

In [ ]:
medals.loc[medals.NOC == 'USA'].groupby('Edition')['Medal'].count()

In [ ]:
# cinsiyeti erkek olanlarin pclass durumlarina gore toplam kurtulan sayilarini cikarmak
titanic.loc[titanic.sex == 'male'].groupby('pclass')['survived'].sum()

In [ ]:
titanic.describe()

In [ ]:
# dataframe deki toplam satir sayisini bulmak
len(titanic)

In [ ]:
titanic.reset_index(drop=True) #drop=True dersek eski indeksi silip yenileri getirir

In [ ]:
# dictionary ve dongu kullanarak bir sutunun degerlerini "isin" ile kontrol etmek, gruplandirmak ve yeniden adlandirmak
replacement = {
    'underground floor': ['Kot 1', 'Kot 2', 'Kot 3', 'Yarı Bodrum', 'Bodrum', 'Bodrum ve Zemin'],
    'ground floor':['1. Kat', 'Bahçe Katı', 'Giriş Katı', 'Villa Katı', 'Yüksek Giriş', 'Zemin'],
    'upper':['10. Kat','11. Kat','12. Kat','13. Kat','14. Kat','15. Kat','16. Kat','17. Kat','18. Kat','19. Kat','20. Kat','21 ve üzeri','2. Kat','3. Kat','4. Kat','5. Kat','6. Kat','7. Kat','8. Kat','9. Kat','Ara Kat'],
    'attic':['Çatı Katı','Teras Katı','En Üst Kat']}
for k,v in replacement.items():
    medal.loc[titanic.floor.isin(v), 'floor'] = k

In [ ]:
# dataframe degerlerini isin ile karsilastirmak. 
titanic2 = pd.DataFrame({'A': [1, 2, 3], 'B': ['a', 'b', 'f']})
other = pd.DataFrame({'A': [1, 3, 3, 2], 'B': ['e', 'f', 'f', 'e']})
titanic2.isin(other)

In [ ]:
# We convert this feature to datetime,
estate['date'] = pd.to_datetime(estate['date'], format=('%d.%m.%Y'))

# data frame de tarih sutununda hangi gunden kac tane oldugunu bulmak
estate.date.dt.weekday_name.value_counts()

In [ ]:
# iki tarih arasinda kac gun oldugunu bulmak 
(estate.date.max()-estate.date.min()).days

In [ ]:
# hangi tarihte kac tane deger oldugunu grafikte gosterme
estate.date.value_counts().sort_index().plot()

In [ ]:
estate.drop(estate[(estate['area-m2']<23.98) | (estate['area-m2']>460.6) ].index) 

In [ ]:
estate['area-m2'].value_counts().sort_index()

In [ ]:
weather1=
       Mean TemperatureF
Month                   
Apr            61.956044
Jan            32.133333
Jul            68.934783
Oct            43.434783
 
# Reindex weather1 using the list year with forward-fill: weather3
year=
['Jan',
 'Feb',
 'Mar',
 'Apr',
 'May',
 'Jun',
 'Jul',
 'Aug',
 'Sep',
 'Oct',
 'Nov',
 'Dec']
weather2 = weather1.reindex(year).ffill()

In [ ]:
# farkli indexleri olan dataframe leri birbiriyle toplama / ekleme. 
# fill_value=0  yazilmazsa ortak olmayan satirlarin karsisinda NaN yazacaktir
bronze.add(silver, fill_value=0).add(gold, fill_value=0) 

In [ ]:
gdp = pd.read_csv('GDP.csv', index_col='DATE', parse_dates=True )
gdp:
              VALUE
DATE               
1947-01-01    243.1
1947-04-01    246.3
1947-07-01    250.1

post2008 = gdp['2008':] # index teki 2008 den sonraki satirlari almak

In [ ]:
# dataframe i baska bir data frame ile carpmak. kur hesabi yaparken...
sp500 = pd.read_csv('sp500.csv', parse_dates=True, index_col='Date')
dollars = sp500[['Open', 'Close']]
exchange = pd.read_csv('exchange.csv', parse_dates=True, index_col='Date')
pounds = dollars.multiply(exchange['GBP/USD'], axis='rows' )

In [ ]:
titanic.sample(3)

In [ ]:
titanic.corr() # sadece sayisal degerleri karsilastirir, iliskiyi bulmaya calisir

In [ ]:
pp.ProfileReport(estate) # profil raporu alma. sutunlarla ilgili ozet bilgiler.

In [ ]:
titanic.isna().sum()

In [ ]:
titanic.fillna('missing') # NaN value ler yerine 'missing' string degerini koyar

In [ ]:
titanic.loc[:, titanic.isnull().any()].head(10) # her hangi bir bos deger bulunan sutunlari listeler.

In [ ]:
titanic.loc[:, titanic.notnull().all()].head(10) # tum degerleri dolu olan sutunlari getiriyor

In [ ]:
titanic.pclass.value_counts()

In [ ]:
estate[estate['price']==14.00].count()

In [ ]:
estate.count()


In [ ]:
from scipy import stats
z = np.abs(stats.zscore(df.area_m2)) #abs mutlak deger demek
df.area_m2[(z>3)].count()
# According to the z score there are too many outliers. But if we look at the data set,
# this is not normal, so we can deal with the outliers as   above.

In [ ]:
estate = estate.drop(estate[estate.room == '4+1'].index) # .index metodu satiri silmek icin gerekli

In [ ]:
estate.info()